In [1]:
!pip install deplacy
!python -m spacy download fr_core_news_sm

     |████████████████████████████████| 14.7MB 3.5MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=6f4e257934a3b6e979c0fcaf2ae5a172b4a3e8ce0ec68257ccc11e2011ceedc3
  Stored in directory: /tmp/pip-ephem-wheel-cache-vli8gjn8/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [2]:
import pkg_resources,imp
imp.reload(pkg_resources)
import spacy
import re
import numpy as np
nlp=spacy.load("fr_core_news_sm")

In [3]:
import nltk
nltk.download('stopwords')
stopword=nltk.corpus.stopwords.words('french')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()

In [6]:
wn=nltk.WordNetLemmatizer()
def tokenize(text):
    tokens = re.split('\W+', text) #W+ means a word character or - can go there
    return tokens
def remove_stopwords(tokenzed_list):
    text=[word for word in tokenzed_list if word not in stopword]
    return text
def lemmatizing(tokenized_text):
    texte=[]
    for elem in tokenized_text:
      texte.append(stemmer.stem(elem))
    return texte
def return_mean_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner la moyenne des vecteurs pour chaque phrase
    return np.mean([(X.vector) for X in doc], axis=0)

In [7]:
def return_word_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le vecteur lié à chaque token
    return [(X.vector) for X in doc]
def return_token(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le texte de chaque token
    return [X.text for X in doc]

In [8]:
import pandas as pd
import numpy as np

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
df_test = pd.read_csv('/content/drive/My Drive/df_test.csv',delimiter=';')

In [43]:
df_test['query'][0]='(machine learning AND test) OR (cat AND du futur) OR (adorer AND dog) OR (du futur AND machine learning)'
df_test['resume'][0]='adorer le machine learning et du futur'

In [44]:
df_test['title'] = [x.lower() for x in df_test.title]

In [45]:
for i in range (len(df_test)):
  df_test['query'][i] = df_test['query'][i].lower()

In [46]:
import nltk
nltk.download('stopwords')
import pandas as pd
import pickle
import time
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))
from nltk.stem import LancasterStemmer
import numpy as np
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
from nltk.stem import SnowballStemmer
def cleandesc(desc):
    sent = desc
    sent = "".join([x.lower() if x.isalpha()  else " " for x in sent])
    Porter=SnowballStemmer('french')
    sent = " ".join([Porter.stem(x) if x.lower() not in stop_words  else "" for x in sent.split()])
    sent = " ".join(sent.split())
    return sent
start_time=time.time()
df_test['title']= [cleandesc(x.title) for x in df_test.itertuples()]
df_test['resume']= [cleandesc(x.resume) for x in df_test.itertuples()]
df_test['query']= [cleandesc(x.query) for x in df_test.itertuples()]
end_time=time.time()
print("total time : {} mn".format((end_time-start_time)/60))

total time : 9.655157725016275e-05 mn


In [48]:
df_test['cat_pertinence']='Inconnu'

In [56]:
pertinence = []
pertinence_j = []
def mots_commun2(df) :
  for j in range (len(df)): # pour chaque ligne du df,
    innov=0
    gest=0
    separator_or = list(df['query'][j].split(' or ')) #on stocke tous les couple de la requete dans une liste separator_or
    list_test = []
    for k in range (len(separator_or)): #pour chacun de ces couples,
      nb_find = 0
      separator_and = list(separator_or[k].split(' and ')) #on stocke tous les mots du couple de la requete dans une liste separator_and
      nb_present = 0
      for i in range (len(separator_and)): #pour tous les mots de la requete,
        #print(separator_and[i])
        if (df['title'][j].find(separator_and[i]) != -1) : #on le cherche dans le titre, et si c'est présent...
          if (i==0):
            innov+=1 #si la position est 0, c'est de l'innovation : on incrémente
          else :
            gest+=1 #si la position est 1, c'est de la gestion : on incrémente
          nb_present = nb_present + 1 #on incrémente le nombre de mots de la requete trouvé 
          list_test.append(separator_and[i]+";") # et on stock le mot trouvé dans le titre dans une liste
        if (df['resume'][j].find(separator_and[i]) != -1) and (df['resume'][j] not in list_test) : #ensuite, on le cherche dans le resumé, et si c'est présent...
          if (i==0):
            innov+=1 
          else :
            gest+=1
          nb_present = nb_present + 1 #on incrémente le nombre de mots trouvés
          list_test.append(separator_and[i]+";") #... on le met dans une liste
          list_test = list(set(list_test))
      str = ' '.join(list_test) #...
      df['mots'][j]=str[:-1] #...on l'ajoute
      #print('le df pour ',k)
      #print(df['mots'])
      if (k > 0) :
        nb_find = len(df_test['mots'][j].split(';'))
        print('on a trouve ',nb_find)
        print('pour le ',k)
        nb = (nb_find / (len(separator_and)+len(separator_or)))*100
        pertinence.append(nb)
        pertinence_j.append(j)
        if (pertinence[k]<pertinence[k-1]): #on cherche à savoir pour ce couple, si le score précédent était meilleur ou pas. Si oui, on prend celui d'avant car c'est un OR.
          pertinence[k]=pertinence[k-1]
          nb = pertinence[k]
      else :
        nb_find = len(df_test['mots'][j].split(';'))
        nb = (nb_find / len(separator_and))*100
        pertinence.append(nb)
        pertinence_j.append(j)
      if (nb>=100) :
        df['cat_pertinence'][j] = 'I&G' #si on a 100%, on a les 2
      elif (nb>=50 and nb<100) :
        df['cat_pertinence'][j] = 'I&G mais pas du meme couple' #à préciser si c'est l'un ou l'autre
      elif (nb<50 and nb>0 and innov>gest):
        df['cat_pertinence'][j] = 'I'
      elif (nb<50 and nb>0 and gest>innov):
        df['cat_pertinence'][j] = 'G'
      else :
        df['cat_pertinence'][j] = 'Aucun'
      #print(nb,'% des mots de la requête apparaissent dans le titre ou dans le résumé.')
      #print('innov ', innov)
      #print('gestion ', gest)
  pertinence_best = pd.DataFrame(columns=['ligne','score'])
  pertinence_best['ligne'] = pertinence_j
  pertinence_best['score'] = pertinence
  print(pertinence_best)

In [57]:
mots_commun2(df_test)

on a trouve  2
pour le  1
on a trouve  3
pour le  2
on a trouve  3
pour le  3
   ligne  score
0      0   50.0
1      0   50.0
2      0   50.0
3      0   50.0
4      1  100.0


In [51]:
df_test

,url,query,title,resume,mots,cat_pertinence
0,https://graylinegroup.com/machine-learning-and...,machin learning and test or cat and futur or a...,machin learning and the new innov paradigm,ador machin learning futur,futur; machin learning; ador,I&G mais pas du meme couple
1,https://graylinegroup.com/machine-learning-and...,machin learning and innov,machin learning and the new innov paradigm,machin learning is very innov it,innov; machin learning,I&G
